---
jupyter: python3
toc: true
toc-depth: 3
number-sections: true
title: Pandas_03_값 삭제, 대체
date: 2021-11-05 00:02
categories: pandas
author: limyj0708
comments:
  giscus:
    repo: quarto-dev/quarto-docs
format:
    html:
        page-layout: full
---

In [56]:
import pandas as pd
import numpy as np
import copy
from IPython.display import display_html, display

In [57]:
def display_multiple_dfs(dfs:list, styles, margin=10):
    display_target = ''
    for each_df in dfs:
        each_df_html = each_df[0].style.set_caption(f'<b>{each_df[1]}</b>').set_table_styles(styles).set_table_attributes(f"style='display:inline;margin:{margin}px'")._repr_html_()
        display_target += each_df_html
    display_html(display_target, raw = True)

In [58]:
styles = [
    {"selector" : "caption", "props" : "text-align:center; font-size:16px"}
]

In [8]:
df = pd.read_parquet('df.parquet', engine='pyarrow') 

- 특정 조건의 값을 삭제하고 싶은 경우에는, 해당 조건의 반대 조건을 걸어서 반환 결과를 사용하는 식으로 처리한다.

# drop : 원하는 행, 열 지우기
```DataFrame.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')```
 
- 특정 레이블의 행이나 열을 제거한다.
- labels : 제거할 index, 레이블 하나 혹은 리스트 (list-like)
- axis : 0이면 행, 1이면 컬럼 대상
- index : labels, axis=0 대신 사용가능
- columns : labels, axis=1 대신 사용가능
- level : MultiIndex일 경우, 어떤 레벨을 제거할 것인지
- inplace : 원본 변경 할 건가요?
- errors : 'ignore'로 세팅하면, 에러 출력 안 하고 존재하는 레이블만 제거한다.

In [426]:
df

,가,나,다,라,마,바,사,아
1,1,4,NaN,0,NaN,2.000000,2.000000,NaN
2,100,4,NaN,0,NaN,20.000000,2.000000,NaN
3,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
4,1,2,3,4,5,1.414214,1.414214,2.44949
5,1,1,1,1,1,1.000000,1.000000,1.00000
6,1,1,1,1,1,1.000000,1.000000,1.00000
7,22,22,22,22,22,22.000000,4.690416,22.00000


In [427]:
df.drop(labels=['가','아'], axis=1)

,나,다,라,마,바,사
1,4,NaN,0,NaN,2.000000,2.000000
2,4,NaN,0,NaN,20.000000,2.000000
3,6,NaN,NaN,NaN,2.449490,2.449490
4,2,3,4,5,1.414214,1.414214
5,1,1,1,1,1.000000,1.000000
6,1,1,1,1,1.000000,1.000000
7,22,22,22,22,22.000000,4.690416


In [428]:
df.drop(labels=[1,7], axis=0)

,가,나,다,라,마,바,사,아
2,100,4,NaN,0,NaN,20.000000,2.000000,NaN
3,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
4,1,2,3,4,5,1.414214,1.414214,2.44949
5,1,1,1,1,1,1.000000,1.000000,1.00000
6,1,1,1,1,1,1.000000,1.000000,1.00000


In [429]:
df.drop(columns=['가','아'])
# df.drop(labels=['가','아'], axis=1)와 같은 결과

,나,다,라,마,바,사
1,4,NaN,0,NaN,2.000000,2.000000
2,4,NaN,0,NaN,20.000000,2.000000
3,6,NaN,NaN,NaN,2.449490,2.449490
4,2,3,4,5,1.414214,1.414214
5,1,1,1,1,1.000000,1.000000
6,1,1,1,1,1.000000,1.000000
7,22,22,22,22,22.000000,4.690416


# Na 대응

## isna : NaN인지 각 값에 대해 확인
- NaN인지 각 값에 대해 확인하여 boolean으로 표현
- isnull() 도 완전히 같은 기능을 한다.
- 왜 같은 기능을 하는 함수가 두 개나 있는지는 아래 링크를 참조
  - [https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas](https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas)
  - This is because pandas' DataFrames are based on R's DataFrames. In R na and null are two separate things. Read [this post](https://www.r-bloggers.com/2010/04/r-na-vs-null/) for more information. 
    However, in python, pandas is built on top of numpy, **which has neither na nor null values. Instead numpy has NaN values (which stands for "Not a Number").** Consequently, pandas also uses NaN values.
- 완전히 반대의 기능을 하는 함수로 notnull()이 있다.
  - [pandas.notnull(obj)](https://pandas.pydata.org/docs/reference/api/pandas.notnull.html)

In [430]:
df.isna()
# 특정 컬럼, 행에 대해서도 사용 가능

,가,나,다,라,마,바,사,아
1,False,False,True,False,True,False,False,True
2,False,False,True,False,True,False,False,True
3,False,False,True,True,True,False,False,True
4,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False


## dropna : NA 드랍
```DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)```

- axis
  - 0 혹은 'index' : missing value가 있는 행을 드랍
  - 1 혹은 'columns' : missing value가 있는 열을 드랍
- how
  - any : missing value가 하나라도 있으면 드랍
  - all : 전체 값이 다 missing value여야 드랍
- thresh : 문턱값. 정수를 입력 시, 정상값이 해당 정수 갯수만큼은 있어야 제거 안 함
- subset : list-like 오브젝트를 넣으면, 해당 index나 컬럼에서만 missing value 체크
- inplace : 원본 변경 할 건가요?

In [431]:
df

,가,나,다,라,마,바,사,아
1,1,4,NaN,0,NaN,2.000000,2.000000,NaN
2,100,4,NaN,0,NaN,20.000000,2.000000,NaN
3,1,6,NaN,NaN,NaN,2.449490,2.449490,NaN
4,1,2,3,4,5,1.414214,1.414214,2.44949
5,1,1,1,1,1,1.000000,1.000000,1.00000
6,1,1,1,1,1,1.000000,1.000000,1.00000
7,22,22,22,22,22,22.000000,4.690416,22.00000


In [432]:
df.dropna() # 기본적으로 행 드랍

,가,나,다,라,마,바,사,아
4,1,2,3,4,5,1.414214,1.414214,2.44949
5,1,1,1,1,1,1.000000,1.000000,1.00000
6,1,1,1,1,1,1.000000,1.000000,1.00000
7,22,22,22,22,22,22.000000,4.690416,22.00000


In [433]:
df.dropna(axis=1) # 열 드랍

,가,나,바,사
1,1,4,2.000000,2.000000
2,100,4,20.000000,2.000000
3,1,6,2.449490,2.449490
4,1,2,1.414214,1.414214
5,1,1,1.000000,1.000000
6,1,1,1.000000,1.000000
7,22,22,22.000000,4.690416


In [434]:
df.dropna(thresh=5) # index 3인 행은 정상값이 4개였음

,가,나,다,라,마,바,사,아
1,1,4,NaN,0,NaN,2.000000,2.000000,NaN
2,100,4,NaN,0,NaN,20.000000,2.000000,NaN
4,1,2,3,4,5,1.414214,1.414214,2.44949
5,1,1,1,1,1,1.000000,1.000000,1.00000
6,1,1,1,1,1,1.000000,1.000000,1.00000
7,22,22,22,22,22,22.000000,4.690416,22.00000


In [435]:
df.dropna(axis=0, subset=['라']) # '라'열만 검사해서 NaN이 있는 행을 제거함

,가,나,다,라,마,바,사,아
1,1,4,NaN,0,NaN,2.000000,2.000000,NaN
2,100,4,NaN,0,NaN,20.000000,2.000000,NaN
4,1,2,3,4,5,1.414214,1.414214,2.44949
5,1,1,1,1,1,1.000000,1.000000,1.00000
6,1,1,1,1,1,1.000000,1.000000,1.00000
7,22,22,22,22,22,22.000000,4.690416,22.00000


## fillna : NaN 데이터 대체하기
`DataFrame.fillna(value=None, method=None, axis=None, inplace=False, limit=None, downcast=None)`

- value : NaN을 무엇으로 채울 것인가?
  - scalar : 0, 1 따위의 값을 넣음
  - dict : {"A": 0, "B": 1, "C": 2, "D": 3}
    - 컬럼 A의 NaN은 0으로, 컬럼 B의 NaN은 1로, 컬럼 C의 NaN은 2로, 컬럼 D의 NaN은 3으로 대체
  - dataframe : 대체 대상 dataframe와 같은 크기의 dataframe을 준비한 후, value에 dataframe을 넣으면 NaN 값만 넣은 dataframe의 값으로 대체된다. 컬럼명이나 인덱스는 원본 dataframe의 것이 유지된다.
- method : 어떤 방법으로 채울까? (value와 같이 사용할 수 없음)
  - backfill, bfill : NaN의 다음 값으로 NaN 채우기.
  - ffill, pad : NaN의 직전 값으로 NaN 채우기.
- axis
  - 0 혹은 'index'
  - 1 혹은 'columns'
- inplace : 원본 변경 할 건가요?
- limit : 위에서부터 NaN 몇 개만 바꿀래? 기본값 None이면 모든 NaN을 바꾸는 것.

In [436]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                  columns=list("ABCD"))
df

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,NaN,3.0,NaN,4


In [437]:
df.fillna(value=0) # 0으로 NaN 채우기

,A,B,C,D
0,0.0,2.0,0.0,0
1,3.0,4.0,0.0,1
2,0.0,0.0,0.0,5
3,0.0,3.0,0.0,4


In [438]:
df.fillna(method='ffill') # NaN의 직전 값으로 NaN 채우기. 'pad'를 써도 마찬가지

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,4.0,NaN,5
3,3.0,3.0,NaN,4


In [439]:
df.fillna(method='bfill') # NaN의 다음 값으로 NaN 채우기. 'backfill'을 써도 마찬가지

,A,B,C,D
0,3.0,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,3.0,NaN,5
3,NaN,3.0,NaN,4


In [440]:
values = {"A": 0, "B": 1, "C": 2, "D": 3}
df.fillna(value=values) # values에 dictionary를 넣어서 컬럼마다 NaN을 다른 값으로 대체

,A,B,C,D
0,0.0,2.0,2.0,0
1,3.0,4.0,2.0,1
2,0.0,1.0,2.0,5
3,0.0,3.0,2.0,4


In [441]:
df.fillna(value=values, limit=1) # limit=1이어서, 최초의 NaN 하나만 대체

,A,B,C,D
0,0.0,2.0,2.0,0
1,3.0,4.0,NaN,1
2,NaN,1.0,NaN,5
3,NaN,3.0,NaN,4


In [442]:
df2 = pd.DataFrame(np.zeros((4, 4)), columns=list("ABCE"))
df2 # 4 by 4 영행렬을 만들어 보자

,A,B,C,E
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0


In [443]:
df.fillna(df2) #원본 df의 컬럼이 유지됨

,A,B,C,D
0,0.0,2.0,0.0,0
1,3.0,4.0,0.0,1
2,0.0,0.0,0.0,5
3,0.0,3.0,0.0,4


In [444]:
df.loc[:,'A'].fillna(df.loc[:,'A'].mean()) # A열의 NaN 값을 A열의 평균으로 채움

0    3.0
1    3.0
2    3.0
3    3.0
Name: A, dtype: float64

# drop_duplicates : 중복값 제거

`DataFrame.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)`

- subset : 컬럼 라벨, 혹은 컬럼 라벨 리스트
  - 넣은 특정 컬럼만 중복값을 체크함. 기본으로는 전체 컬럼의 값이 다 같아야 제거
- keep
  - first : 첫 번째 등장한 것을 제외하면 다 제거
  - last : 마지막에 등장한 것을 제외하면 다 제거
  - False : 몽땅 다 제거
- inplace : 원본 변경 할 건가요?
- ignore_index : True 값을 넣으면, 결과값의 인덱스를 0, 1, ... n-1로 라벨링함

In [445]:
df = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, 3.5, 15, 5]
})
df

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


In [446]:
df.drop_duplicates() # 모든 열의 값이 다 같으면 제거

,brand,style,rating
0,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


In [447]:
df.drop_duplicates(subset=['brand']) # brand 컬럼 하나에서만 값이 같아도 제거

,brand,style,rating
0,Yum Yum,cup,4.0
2,Indomie,cup,3.5


In [448]:
df.drop_duplicates(subset=['brand', 'style'], keep='last')
# brand, style 모두 같으면, 마지막 값만 남김

,brand,style,rating
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
4,Indomie,pack,5.0
